In [14]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats as st
import glob
import warnings

In [15]:
# Задачи для рефакторинга:
#  - Использовать фрейм os или glob для нормального импорта. Done! 
#  - Нормально оформить заголовки и комменты. In process
#  - Обработать ошибки Try/Except In process
# Прогресс - 99% (остался косметический и кодовый рефакторинг)

## Описание проекта

#### Вы аналитик компании «Мегалайн» — федерального оператора сотовой связи. Клиентам предлагают два тарифного плана: «Смарт» и «Ультра». Чтобы скорректировать рекламный бюджет, коммерческий департамент хочет понять, какой тариф приносит больше денег. 

#### Вам предстоит сделать предварительный анализ тарифов на небольшой выборке клиентов. В вашем распоряжении данные 500 пользователей «Мегалайна»: кто они, откуда, каким тарифом пользуются, сколько звонков и сообщений каждый отправил за 2018 год. Нужно проанализировать поведение клиентов и сделать вывод — какой тариф лучше.

> Описание проекта и датасеты взяты из одного из проектов YandexPracticum*

In [16]:
path = '/home/zachary/data_analysis_dir/data_analysis_env/files_by_yandex_practicum/'
all_files = glob.glob(path + "/*.csv")
all_files

['/home/zachary/data_analysis_dir/data_analysis_env/files_by_yandex_practicum/calls.csv',
 '/home/zachary/data_analysis_dir/data_analysis_env/files_by_yandex_practicum/internet.csv',
 '/home/zachary/data_analysis_dir/data_analysis_env/files_by_yandex_practicum/users.csv',
 '/home/zachary/data_analysis_dir/data_analysis_env/files_by_yandex_practicum/tariffs.csv',
 '/home/zachary/data_analysis_dir/data_analysis_env/files_by_yandex_practicum/messages.csv']

In [17]:
calls, internet, users, tariffs, messages = [pd.read_csv(i) for i in all_files]

## Предобработка данных и первичный анализ:

In [18]:
# Меняем типы данных в подозрительных датасетах
calls['call_date'] = calls['call_date'].apply(pd.to_datetime)
internet['session_date'] = internet['session_date'].apply(pd.to_datetime)
messages['message_date'] = messages['message_date'].apply(pd.to_datetime)

#### Посчитайте для каждого пользователя:
<br>
1. количество сделанных звонков и израсходованных минут разговора по месяцам; <br> 2. количество отправленных сообщений по месяцам; <br> 3. объем израсходованного интернет-трафика по месяцам; <br> 4. помесячную выручку с каждого пользователя  <br> (вычтите бесплатный лимит из суммарного количества звонков, <br> сообщений и интернет-трафика; остаток умножьте на значение из тарифного плана; прибавьте абонентскую плату,  <br> соответствующую тарифному плану).

In [19]:
# Создание столбцов с месяцами в каждом датасете
calls['month'] = calls['call_date'].dt.strftime('%b')
internet['month'] = internet['session_date'].dt.strftime('%b')
messages['month'] = messages['message_date'].dt.strftime('%b')

In [20]:
# 1. количество сделанных звонков и израсходованных минут разговора по месяцам
calls_filtred = calls.groupby(by=['user_id', 'month'], as_index=False).aggregate({'duration': ['sum', 'count']})
calls_filtred['duration'] = calls_filtred['duration'].apply(np.ceil)
calls_filtred.head()

user_id month duration      
                     sum count
0    1000   Aug    390.0  52.0
1    1000   Dec    313.0  46.0
2    1000   Jul    319.0  47.0
3    1000   Jun    159.0  43.0
4    1000   May    151.0  22.0

In [21]:
# 2. количество отправленных сообщений по месяцам;
messages_filtered = messages.groupby(by=['user_id', 'month'], as_index=False).aggregate(number_of_messages=('id', 'count'))
messages_filtered.head()

,user_id,month,number_of_messages
0,1000,Aug,81
1,1000,Dec,70
2,1000,Jul,75
3,1000,Jun,60
4,1000,May,22


In [22]:
# 3. объем израсходованного интернет-трафика по месяцам;
internet_filtered = internet.groupby(by=['user_id', 'month'], as_index=False).aggregate({'mb_used': 'sum'})
internet_filtered.head()

,user_id,month,mb_used
0,1000,Aug,14055.93
1,1000,Dec,9817.61
2,1000,Jul,14003.64
3,1000,Jun,23233.77
4,1000,May,2253.49


In [23]:
# Дабы избежать путаницы, смерджим тарифы к каждому из трех датасетов для дальнейших рассчетов
warnings.filterwarnings("error")
try:
    calls_filtred = pd.merge(calls_filtred, users[['user_id', 'tariff']], on='user_id')
    messages_filtered = pd.merge(messages_filtered, users[['user_id', 'tariff']], on='user_id')
    internet_filtered = pd.merge(internet_filtered, users[['user_id', 'tariff']], on='user_id')
except RuntimeWarning:
    import ipdb; ipdb.set_trace()

FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left,1 on the right)

In [ ]:
# Изменим имена столбцов для лучшей читабельности (?)
calls_filtred.set_axis(['user_id', 'drop', 'month', 'call_duration_sum', 'call_duration_count', 'tariff'],
                       axis='columns', 
                       inplace=True)
calls_filtred.head()

In [ ]:
def count_calls(row):

    tariff = row['tariff']
    duration = row['call_duration_sum']
    
    if tariff == 'ultra':
        if duration > 1950:
            return duration - 1950
        else:
            return 0
    else: 
        if duration > 550:
            return (duration - 500) * 3
        else:
            return 0

def count_messages(row):
    
    tariff = row['tariff']
    quantity_of_messages = row['number_of_messages']
    
    if tariff == 'ultra':
        if quantity_of_messages > 1000:
            return quantity_of_messages - 1000 
        else:
            return 0
    else:
        if quantity_of_messages > 50:
            return (quantity_of_messages - 50) * 3
        else: 
            return 0

def count_internet(row):
    
    tariff = row['tariff']
    sum_mb_per_month = row['mb_used']
    
    if tariff == 'ultra':
        if sum_mb_per_month > 30720:
            return np.ceil((sum_mb_per_month - 30720) / 1024) * 150
        else: 
            return 0
    else:
        if sum_mb_per_month > 15360:
            return np.ceil((sum_mb_per_month - 15360) / 1024) * 200
        else: 
            return 0    

In [ ]:
calls_filtred['calls_over_tariff'] = calls_filtred.apply(count_calls, axis=1)

In [ ]:
messages_filtered['messages_over_tariff'] = messages_filtered.apply(count_messages, axis=1)

In [ ]:
internet_filtered['internet_over_tariff'] = internet_filtered.apply(count_internet, axis=1)

In [ ]:
df_merged = internet_filtered.merge(calls_filtred) \
                             .merge(messages_filtered)
df_merged.head()

In [ ]:
tariffs

In [ ]:
# 4. Помесячная выручка
def total_revenue(row):
    """Суммировать отклонения от стоимости тарифа и сам тариф, 
    таким образом получим помесячную выручку для каждого пользователя"""
    tariff = row['tariff']
    calls = row['calls_over_tariff']
    messages = row['messages_over_tariff']
    internet = row['internet_over_tariff']
    
    if tariff == 'ultra':
        return calls + messages + internet + 1950
    else:
        return calls + messages + internet + 550

In [ ]:
df_merged['attempt_of_revenue'] = df_merged.apply(total_revenue, axis=1)
# Избавимся от дубликатов столбцов
df_merged.drop(columns='drop', inplace = True)

In [ ]:
df_merged.head()

## Анализ обработанных данных:

In [ ]:
# Сколько минут разговора, сколько сообщений и какой объём интернет-трафика 
# требуется пользователям каждого тарифа в месяц?
# Посчитайте среднее количество, дисперсию и стандартное отклонение. Постройте гистограммы. Опишите распределения.

In [ ]:
# - Для тарифа smart:
average_per_month_smart = df_merged[df_merged['tariff'] == 'smart'] \
                          .groupby(by='user_id') \
                          .aggregate(average_of_calls=('call_duration_count', 'mean'), 
                                      average_of_messages=('number_of_messages', 'mean'),
                                      average_of_mb_used=('mb_used', 'mean'),
                                      revenue=('attempt_of_revenue', 'mean'))
average_per_month_smart.head()

In [ ]:
# Построим гистограммы и найдем меры изменчивости(?)
try:
    fig = plt.figure(figsize=(14,18))

    a, b, c = 3, 2, 1
    for name in average_per_month_smart.columns:
        plt.subplot(a, b, c)
        plt.title(f"{name}: histplot, subplot: {a}{b}{c}")
        plt.xlabel(name)
        sns.histplot(average_per_month_smart[name], kde=True)
        c = c + 1

        plt.subplot(a, b, c)
        plt.title(f"{name}: boxplot, subplot: {a}{b}{c}")
        plt.xlabel(name)
        plt.boxplot(average_per_month_smart[name])
        c = c + 1
except ValueError:
    
    plt.show()

##### - В целом, наши пользователи тарифа смарт делают около 60 звонков в месяц. <br> - По количеству сообщений не все так однозначно, т.к. имеется скошенность вправо и явные выбросы, <br> поэтому стоит доверять медиане - 33 сообщениям, как устойчивой к выбросам величине. <br> - Трафик также, как и звонки распределен ближе к нормальному, а значит мы можем смело утверждать, <br> что в среднем тратится около 16.3 гб

In [ ]:
average_per_month_smart.describe()

In [ ]:
# - Для тарифа ultra:
average_per_month_ultra = df_merged[df_merged['tariff'] == 'ultra'] \
                          .groupby(by='user_id') \
                          .aggregate(average_of_calls=('call_duration_count', 'mean'), 
                                      average_of_messages=('number_of_messages', 'mean'),
                                      average_of_mb_used=('mb_used', 'mean'))
average_per_month_ultra.describe()

In [ ]:
# Построим гистограммы и найдем меры изменчивости(?)
try:
    fig = plt.figure(figsize=(14,18))

    a, b, c = 3, 2, 1
    for name in average_per_month_ultra.columns:
        plt.subplot(a, b, c)
        plt.title(f"{name}: histplot, subplot: {a}{b}{c}")
        plt.xlabel(name)
        sns.histplot(average_per_month_ultra[name], kde=True)
        c = c + 1

        plt.subplot(a, b, c)
        plt.title(f"{name}: boxplot, subplot: {a}{b}{c}")
        plt.xlabel(name)
        plt.boxplot(average_per_month_ultra[name])
        c = c + 1
        
except ValueError:
    
    plt.show()


#### - Несмотря на небольшую выборку, мы можем точно сказать о правдивости данных, т.е звонки распределены нормально <br> и в среднем составляют чуть больше 77 в месяц <br> - Что касается сообщений, они также имеют небольшую скошенность вправо и соответственно и выбросы, <br> а значит, стоит доверять медиане - 59 в месяц <br> - Использовано траффика на 19.6 гб, что на ~4 гб больше, чем на тарифе смарт

In [ ]:
average_per_month_ultra.describe()

## Проверка Гипотез

In [ ]:
# 1. средняя выручка пользователей тарифов «Ультра» и «Смарт» различаются
average_revenue_smart = df_merged[df_merged['tariff'] == 'smart'] \
                          .groupby(by='user_id') \
                          .aggregate(revenue=('attempt_of_revenue', 'mean'))
average_revenue_ultra = df_merged[df_merged['tariff'] == 'ultra'] \
                          .groupby(by='user_id') \
                          .aggregate(revenue=('attempt_of_revenue', 'mean'))

first_test_result = st.ttest_ind(average_revenue_smart['revenue'],
                                 average_revenue_ultra['revenue'], 
                                 equal_var=True).pvalue
alpha = 0.05
print(f"p-значение: {first_test_result}")

if first_test_result < alpha: 
    print('Отвергаем нулевую гипотезу!\nP value слишком низкий, что и подтверждает наше предположение')
else: 
    print('Не получилось отвергнуть нулевую гипотезу!')

In [ ]:
# 2. Выручка пользователей из Москвы отличается от выручки пользователей из других регионов
average_revenue_moscow = pd.merge(df_merged, users[['user_id', 'city']]).query("city == 'Москва'") \
                           .groupby(by='user_id') \
                           .aggregate(revenue=('attempt_of_revenue', 'mean'))
average_revenue_all_region = pd.merge(df_merged, users[['user_id', 'city']]) \
                               .query("city != 'Москва'") \
                               .groupby(by='user_id').aggregate(revenue=('attempt_of_revenue', 'mean'))

second_test_result = st.ttest_ind(average_revenue_moscow['revenue'], 
                                  average_revenue_all_region['revenue'], 
                                  equal_var=True).pvalue

print(f"p-значение: {second_test_result}")

if second_test_result < alpha: 
    print('Отвергаем нулевую гипотезу!')
else: 
    print('Не получилось отвергнуть нулевую гипотезу!\nВыручка слабо зависит от региона')

#### Гипотезы нам показали, что Региональных факторов и Москвы, влияющих на выручку - нет, однако выручка зависит от тарифа. <br> Для того, чтобы понять какой тариф лучше, мы можем вызвать функцию describe(), либо сравнить Меры Центральной Тенденции:

In [ ]:
# Победитель - тариф ультра.
print(f"Median.\n Ultra: {average_revenue_ultra['revenue'].median()} comparison to Smart: {average_revenue_smart['revenue'].median()}")
print(f"Mean.\n Ultra: {average_revenue_ultra['revenue'].mean()} comparison to Smart: {average_revenue_smart['revenue'].mean()}")